In [108]:
import pandas as pd
import os
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as ImbPipeline

In [110]:
train_data = pd.read_csv('/Users/marlenawasiak/Desktop/Data_Collection/UNSW_NB15_training-set.csv')
test_data = pd.read_csv('/Users/marlenawasiak/Desktop/Data_Collection/UNSW_NB15_testing-set.csv')

In [111]:
X_train = train_data.drop(columns=['attack_cat'])
y_train = train_data['attack_cat']

X_test = test_data.drop(columns=['attack_cat'])
y_test = test_data['attack_cat']

# Encode categorical features
categorical_features = ['proto', 'service', 'state']
X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)

# Align test set columns with training set columns (fill missing columns with 0)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Encode the target label (attack category) in both training and test sets
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Apply oversampling to the training set
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Scale the training and test sets
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)  # Fit and transform on training set
X_test_scaled = scaler.transform(X_test)                     # Only transform the test set


In [113]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

# Define the neural network model
# Experiment with a deeper model
model = Sequential()
model.add(Dense(128, input_dim=X_train_resampled.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_resampled, y_train_resampled, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test), class_weight=class_weight_dict)


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 592us/step - accuracy: 0.6430 - loss: 0.8994 - val_accuracy: 0.6394 - val_loss: 1.3868
Epoch 2/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 622us/step - accuracy: 0.7240 - loss: 0.6669 - val_accuracy: 0.6317 - val_loss: 2.1254
Epoch 3/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 635us/step - accuracy: 0.7337 - loss: 0.6376 - val_accuracy: 0.7726 - val_loss: 1.6026
Epoch 4/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 601us/step - accuracy: 0.7381 - loss: 0.6215 - val_accuracy: 0.7698 - val_loss: 1.8644
Epoch 5/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 652us/step - accuracy: 0.7421 - loss: 0.6072 - val_accuracy: 0.7093 - val_loss: 2.5264
Epoch 6/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 644us/step - accuracy: 0.7455 - loss: 0.5999 - val_accuracy: 0.7764 - val_loss: 2.3490
Epoch 7/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 639us/step - accuracy: 0.7471 - loss: 0.5928 - val_accuracy: 0.7909 - val_loss: 2.7585
Epoch 8/50
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 654us/s

In [114]:
# Evaluate the model
y_pred_encoded = model.predict(X_test_scaled).argmax(axis=1)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred_encoded)
print("Test Accuracy:", test_accuracy)

# Generate classification report with specified labels to handle all classes
class_report = classification_report(
    y_test, 
    y_pred_encoded, 
    labels=range(len(label_encoder.classes_)), 
    target_names=label_encoder.classes_, 
    zero_division=1
)
print("Classification Report - Clean Data:\n", class_report)


2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 301us/step
Test Accuracy: 0.6788976339697809
Classification Report - Clean Data:
                 precision    recall  f1-score   support

      Analysis       0.06      0.21      0.09       677
      Backdoor       0.05      0.55      0.09       583
           DoS       0.25      0.11      0.15      4089
      Exploits       0.35      0.50      0.41     11132
       Fuzzers       0.49      0.64      0.55      6062
       Generic       1.00      0.33      0.50     18871
        Normal       1.00      1.00      1.00     37000
Reconnaissance       0.58      0.62      0.60      3496
     Shellcode       0.20      0.43      0.27       378
         Worms       0.12      0.20      0.15        44

      accuracy                           0.68     82332
     macro avg       0.41      0.46      0.38     82332
  weighted avg       0.80      0.68      0.70     82332



In [118]:
import numpy as np
import tensorflow as tf
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, CarliniL2Method
from art.estimators.classification import TensorFlowV2Classifier
from sklearn.metrics import accuracy_score, classification_report

# Critical features from SHAP analysis
critical_features = [40, 174, 159, 31, 25]

# Create a mask to focus on the critical features
mask = np.zeros(X_train_resampled.shape[1], dtype=np.float32)
mask[critical_features] = 1.0

# Define the ART classifier
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=len(label_encoder.classes_),
    input_shape=(X_train_resampled.shape[1],),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
)

# Apply FGSM attack
print("\n--- FGSM Attack ---")
fgsm = FastGradientMethod(estimator=classifier, eps=3.0, targeted=False)
X_test_adv_fgsm = fgsm.generate(X_test_scaled, mask=mask)
fgsm_preds = model.predict(X_test_adv_fgsm).argmax(axis=1)
fgsm_accuracy = accuracy_score(y_test, fgsm_preds)
fgsm_class_report = classification_report(y_test, fgsm_preds, target_names=label_encoder.classes_, zero_division=1)
print(f"FGSM Adversarial Accuracy: {fgsm_accuracy}")
print(f"Classification Report on FGSM Adversarial Examples:\n{fgsm_class_report}")




--- FGSM Attack ---
2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 384us/step
FGSM Adversarial Accuracy: 0.50010931351115
Classification Report on FGSM Adversarial Examples:
                precision    recall  f1-score   support

      Analysis       0.02      0.00      0.01       677
      Backdoor       0.04      0.30      0.08       583
           DoS       0.05      0.07      0.05      4089
      Exploits       0.40      0.19      0.25     11132
       Fuzzers       0.11      0.14      0.12      6062
       Generic       0.73      0.03      0.07     18871
        Normal       0.74      1.00      0.85     37000
Reconnaissance       0.02      0.01      0.02      3496
     Shellcode       0.02      0.30      0.04       378
         Worms       0.00      0.00      0.00        44

      accuracy                           0.50     82332
     macro avg       0.21      0.20      0.15     82332
  weighted avg       0.56      0.50      0.44     82332



In [120]:
# Apply PGD attack with dynamic feature masking
print("\n--- PGD Attack ---")
pgd = ProjectedGradientDescent(estimator=classifier, eps=2.0, eps_step=0.1, max_iter=100, targeted=False)
X_test_adv_pgd = pgd.generate(X_test_scaled, mask=mask)
pgd_preds = model.predict(X_test_adv_pgd).argmax(axis=1)
pgd_accuracy = accuracy_score(y_test, pgd_preds)
pgd_class_report = classification_report(y_test, pgd_preds, target_names=label_encoder.classes_, zero_division=1)
print(f"PGD Adversarial Accuracy: {pgd_accuracy}")
print(f"Classification Report on PGD Adversarial Examples:\n{pgd_class_report}")



--- PGD Attack ---


PGD - Batches: 0it [00:00, ?it/s]

2024-11-17 23:54:09.603669: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 258us/step
PGD Adversarial Accuracy: 0.5566608366127387
Classification Report on PGD Adversarial Examples:
                precision    recall  f1-score   support

      Analysis       0.01      0.06      0.02       677
      Backdoor       0.04      0.54      0.07       583
           DoS       0.07      0.10      0.09      4089
      Exploits       0.35      0.16      0.22     11132
       Fuzzers       0.12      0.13      0.12      6062
       Generic       0.95      0.27      0.42     18871
        Normal       0.89      1.00      0.94     37000
Reconnaissance       0.13      0.13      0.13      3496
     Shellcode       0.02      0.16      0.03       378
         Worms       0.02      0.11      0.03        44

      accuracy                           0.56     82332
     macro avg       0.26      0.27      0.21     82332
  weighted avg       0.68      0.56      0.57     82332



In [83]:
print("\n--- Carlini & Wagner Attack (on a subset) ---")

# Select a subset that includes multiple classes
subset_indices = []
classes_to_include = np.unique(y_test)
for cls in classes_to_include:
    indices = np.where(y_test == cls)[0][:50]  # Take up to 50 samples per class
    subset_indices.extend(indices)

# Create a subset based on the selected indices
subset_indices = np.array(subset_indices)
X_test_subset = X_test_scaled[subset_indices]
y_test_subset = y_test_encoded[subset_indices]

# Run Carlini & Wagner attack on the subset
cw = CarliniL2Method(classifier=classifier, confidence=1.0, targeted=False, max_iter=100)
X_test_adv_cw_subset = cw.generate(X_test_subset, mask=mask)

# Evaluate the attack
cw_preds_subset = mlp_model.predict(X_test_adv_cw_subset).argmax(axis=1)
cw_accuracy_subset = accuracy_score(y_test_subset, cw_preds_subset)
cw_class_report_subset = classification_report(
    y_test_subset, 
    cw_preds_subset, 
    target_names=label_encoder.classes_, 
    zero_division=1
)

print(f"Carlini & Wagner Adversarial Accuracy (Subset): {cw_accuracy_subset}")
print(f"\nClassification Report on Carlini & Wagner Adversarial Examples (Subset):\n{cw_class_report_subset}")
results = {
    "clean": {
        "accuracy": clean_accuracy,
        "report": clean_class_report
    },
    "fgsm": {
        "accuracy": fgsm_accuracy,
        "report": fgsm_class_report
    },
    "pgd": {
        "accuracy": pgd_accuracy,
        "report": pgd_class_report
}
}
# Save results for C&W attack on the subset
results["cw_subset"] = {
    "accuracy": cw_accuracy_subset,
    "report": cw_class_report_subset
}

np.save("dynamic_feature_mask.npy", mask)



--- Carlini & Wagner Attack (on a subset) ---


C&W L_2:   0%|          | 0/494 [00:00<?, ?it/s]

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 362us/step
Carlini & Wagner Adversarial Accuracy (Subset): 0.20850202429149797

Classification Report on Carlini & Wagner Adversarial Examples (Subset):
                precision    recall  f1-score   support

      Analysis       0.00      0.00      0.00        50
      Backdoor       0.15      0.92      0.26        50
           DoS       0.00      0.00      0.00        50
      Exploits       0.09      0.02      0.03        50
       Fuzzers       0.41      0.18      0.25        50
       Generic       0.00      0.00      0.00        50
        Normal       0.31      0.92      0.46        50
Reconnaissance       0.00      0.00      0.00        50
     Shellcode       1.00      0.00      0.00        50
         Worms       1.00      0.02      0.04        44

      accuracy                           0.21       494
     macro avg       0.30      0.21      0.11       494
  weighted avg       0.29      0.21      0.11       494



In [86]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
from sklearn.preprocessing import QuantileTransformer
# Assuming train_data and test_data are available
X_train = train_data.drop(columns=['attack_cat'])
y_train = train_data['attack_cat']
X_test = test_data.drop(columns=['attack_cat'])
y_test = test_data['attack_cat']

# Step 1: One-hot encode categorical features
categorical_features = ['proto', 'service', 'state']
X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Step 2: Encode the target labels (attack category)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Step 3: Handle class imbalance with oversampling
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train_encoded)

# Step 4: Scale features using StandardScaler and QuantileTransformer
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
X_train_scaled = quantile_transformer.fit_transform(X_train_scaled)
X_test_scaled = quantile_transformer.transform(X_test_scaled)

# Step 5: Define and compile the MLP model
num_classes = len(np.unique(y_train_encoded))
mlp_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')  # Ensure the output layer matches the number of classes
])

mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 6: Train the model
history = mlp_model.fit(X_train_scaled, y_train_resampled, epochs=30, batch_size=32, validation_data=(X_test_scaled, y_test_encoded))

Epoch 1/30


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


17500/17500 ━━━━━━━━━━━━━━━━━━━━ 9s 497us/step - accuracy: 0.6358 - loss: 0.9265 - val_accuracy: 0.8372 - val_loss: 0.4638
Epoch 2/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 9s 492us/step - accuracy: 0.7053 - loss: 0.7201 - val_accuracy: 0.8332 - val_loss: 0.4546
Epoch 3/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 9s 528us/step - accuracy: 0.7154 - loss: 0.6920 - val_accuracy: 0.8371 - val_loss: 0.4342
Epoch 4/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 471us/step - accuracy: 0.7206 - loss: 0.6757 - val_accuracy: 0.7945 - val_loss: 0.6712
Epoch 5/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 466us/step - accuracy: 0.7279 - loss: 0.6601 - val_accuracy: 0.8250 - val_loss: 0.9787
Epoch 6/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 9s 496us/step - accuracy: 0.7310 - loss: 0.6489 - val_accuracy: 0.8391 - val_loss: 0.5680
Epoch 7/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 9s 502us/step - accuracy: 0.7290 - loss: 0.6483 - val_accuracy: 0.8346 - val_loss: 0.8540
Epoch 8/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 483us/step - accuracy: 0.7

In [89]:
# Step 7: Evaluate the model on clean data
y_pred = mlp_model.predict(X_test_scaled).argmax(axis=1)
clean_accuracy = accuracy_score(y_test_encoded, y_pred)

# Step 8: Decode predictions for a report with original class labels
y_test_labels = label_encoder.inverse_transform(y_test_encoded)
y_pred_labels = label_encoder.inverse_transform(y_pred)

# Classification report and confusion matrix with decoded labels
decoded_class_report = classification_report(y_test_labels, y_pred_labels, zero_division=1)
decoded_conf_matrix = confusion_matrix(y_test_labels, y_pred_labels)

# Print the results
print("MLP Model Accuracy on Clean Data:", clean_accuracy)
print("\nClassification Report on Clean Data with Attack Names:\n", decoded_class_report)
print("\nConfusion Matrix on Clean Data with Attack Names:\n", decoded_conf_matrix)


2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 277us/step
MLP Model Accuracy on Clean Data: 0.8394427440120488

Classification Report on Clean Data with Attack Names:
                 precision    recall  f1-score   support

      Analysis       0.00      0.00      0.00       677
      Backdoor       0.06      0.96      0.12       583
           DoS       0.33      0.07      0.11      4089
      Exploits       0.85      0.52      0.64     11132
       Fuzzers       0.86      0.68      0.76      6062
       Generic       1.00      0.96      0.98     18871
        Normal       1.00      1.00      1.00     37000
Reconnaissance       0.84      0.85      0.84      3496
     Shellcode       0.21      0.87      0.34       378
         Worms       0.08      0.77      0.15        44

      accuracy                           0.84     82332
     macro avg       0.52      0.67      0.49     82332
  weighted avg       0.91      0.84      0.86     82332


Confusion Matrix on Clean Data with Attack Names:
 [[    

In [87]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, QuantileTransformer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent, CarliniL2Method
from art.estimators.classification import TensorFlowV2Classifier

# Assuming train_data and test_data are available
X_train = train_data.drop(columns=['attack_cat'])
y_train = train_data['attack_cat']
X_test = test_data.drop(columns=['attack_cat'])
y_test = test_data['attack_cat']

# Step 1: One-hot encode categorical features
categorical_features = ['proto', 'service', 'state']
X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_features, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Step 2: Encode the target labels (attack category)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Step 3: Handle class imbalance with oversampling
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train_encoded)

# Step 4: Scale features 
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=42)
X_train_scaled = quantile_transformer.fit_transform(X_train_scaled)
X_test_scaled = quantile_transformer.transform(X_test_scaled)

# Step 5: Define and compile the MLP model
num_classes = len(np.unique(y_train_encoded))
mlp_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

mlp_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 6: Train the model
history = mlp_model.fit(X_train_scaled, y_train_resampled, epochs=30, batch_size=32, validation_data=(X_test_scaled, y_test_encoded))

# Step 7: Evaluate the model on clean data
y_pred = mlp_model.predict(X_test_scaled).argmax(axis=1)
clean_accuracy = accuracy_score(y_test_encoded, y_pred)
clean_class_report = classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_, zero_division=1)
print("MLP Model Accuracy on Clean Data:", clean_accuracy)
print("\nClassification Report on Clean Data:\n", clean_class_report)

# Define critical features based on SHAP 
critical_features = [40, 174, 159, 31, 25] 
mask = np.zeros(X_train_scaled.shape[1], dtype=np.float32)
mask[critical_features] = 1.0

# Define the ART classifier
classifier = TensorFlowV2Classifier(
    model=mlp_model,
    nb_classes=num_classes,
    input_shape=(X_train_scaled.shape[1],),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy()
)

# Step 8: FGSM Attack
print("\n--- FGSM Attack ---")
fgsm = FastGradientMethod(estimator=classifier, eps=3.0, targeted=False)
X_test_adv_fgsm = fgsm.generate(X_test_scaled, mask=mask)
fgsm_preds = mlp_model.predict(X_test_adv_fgsm).argmax(axis=1)
fgsm_accuracy = accuracy_score(y_test_encoded, fgsm_preds)
fgsm_class_report = classification_report(y_test_encoded, fgsm_preds, target_names=label_encoder.classes_, zero_division=1)
print(f"FGSM Adversarial Accuracy: {fgsm_accuracy}")
print(f"\nClassification Report on FGSM Adversarial Examples:\n{fgsm_class_report}")




/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 580us/step - accuracy: 0.6303 - loss: 0.9373 - val_accuracy: 0.8200 - val_loss: 0.5085
Epoch 2/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 465us/step - accuracy: 0.7055 - loss: 0.7171 - val_accuracy: 0.8379 - val_loss: 0.4307
Epoch 3/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 10s 550us/step - accuracy: 0.7137 - loss: 0.6931 - val_accuracy: 0.8362 - val_loss: 0.4520
Epoch 4/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 466us/step - accuracy: 0.7200 - loss: 0.6781 - val_accuracy: 0.8289 - val_loss: 0.4996
Epoch 5/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 628us/step - accuracy: 0.7246 - loss: 0.6651 - val_accuracy: 0.6620 - val_loss: 1.9659
Epoch 6/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 11s 621us/step - accuracy: 0.7272 - loss: 0.6580 - val_accuracy: 0.6289 - val_loss: 3.8935
Epoch 7/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 9s 509us/step - accuracy: 0.7292 - loss: 0.6518 - val_accuracy: 0.6236 - val_loss: 5.5518
Epoch 8/30
17500/17500 ━━━━━━━━━━━━━━━━━━━━ 8s 484us/step 

In [95]:
# Step 9: PGD Attack
print("\n--- PGD Attack ---")
pgd = ProjectedGradientDescent(estimator=classifier, eps=3.0, eps_step=0.1, max_iter=100, targeted=False)
X_test_adv_pgd = pgd.generate(X_test_scaled, mask=mask)
pgd_preds = mlp_model.predict(X_test_adv_pgd).argmax(axis=1)
pgd_accuracy = accuracy_score(y_test_encoded, pgd_preds)
pgd_class_report = classification_report(y_test_encoded, pgd_preds, target_names=label_encoder.classes_, zero_division=1)
print(f"PGD Adversarial Accuracy: {pgd_accuracy}")
print(f"\nClassification Report on PGD Adversarial Examples:\n{pgd_class_report}")
print("\n--- Carlini & Wagner Attack (on a subset) ---")



--- PGD Attack ---


PGD - Batches: 0it [00:00, ?it/s]

2573/2573 ━━━━━━━━━━━━━━━━━━━━ 1s 261us/step
PGD Adversarial Accuracy: 0.6453262401010542

Classification Report on PGD Adversarial Examples:
                precision    recall  f1-score   support

      Analysis       0.03      0.13      0.05       677
      Backdoor       0.06      0.90      0.12       583
           DoS       0.13      0.10      0.11      4089
      Exploits       0.69      0.25      0.37     11132
       Fuzzers       0.55      0.25      0.34      6062
       Generic       0.99      0.52      0.68     18871
        Normal       0.98      1.00      0.99     37000
Reconnaissance       0.09      0.24      0.13      3496
     Shellcode       0.05      0.43      0.10       378
         Worms       0.02      0.32      0.03        44

      accuracy                           0.65     82332
     macro avg       0.36      0.41      0.29     82332
  weighted avg       0.81      0.65      0.69     82332


--- Carlini & Wagner Attack (on a subset) ---


In [97]:

# Select a subset that includes multiple classes
subset_indices = []
classes_to_include = np.unique(y_test_encoded)
for cls in classes_to_include:
    indices = np.where(y_test_encoded == cls)[0][:50]  # Take up to 50 samples per class
    subset_indices.extend(indices)

# Create a subset based on the selected indices
subset_indices = np.array(subset_indices)
X_test_subset = X_test_scaled[subset_indices]
y_test_subset = y_test_encoded[subset_indices]

# Run Carlini & Wagner attack on the subset
cw = CarliniL2Method(classifier=classifier, confidence=1.0, targeted=False, max_iter=100)
X_test_adv_cw_subset = cw.generate(X_test_subset, mask=mask)

# Evaluate the attack
cw_preds_subset = mlp_model.predict(X_test_adv_cw_subset).argmax(axis=1)
cw_accuracy_subset = accuracy_score(y_test_subset, cw_preds_subset)
cw_class_report_subset = classification_report(
    y_test_subset, 
    cw_preds_subset, 
    target_names=label_encoder.classes_, 
    zero_division=1
)

print(f"Carlini & Wagner Adversarial Accuracy (Subset): {cw_accuracy_subset}")
print(f"\nClassification Report on Carlini & Wagner Adversarial Examples (Subset):\n{cw_class_report_subset}")

C&W L_2:   0%|          | 0/494 [00:00<?, ?it/s]

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step
Carlini & Wagner Adversarial Accuracy (Subset): 0.631578947368421

Classification Report on Carlini & Wagner Adversarial Examples (Subset):
                precision    recall  f1-score   support

      Analysis       0.00      0.00      0.00        50
      Backdoor       0.28      0.86      0.42        50
           DoS       0.57      0.08      0.14        50
      Exploits       0.60      0.36      0.45        50
       Fuzzers       0.76      0.96      0.85        50
       Generic       1.00      0.96      0.98        50
        Normal       0.83      1.00      0.91        50
Reconnaissance       0.84      0.64      0.73        50
     Shellcode       0.77      0.72      0.74        50
         Worms       0.97      0.75      0.85        44

      accuracy                           0.63       494
     macro avg       0.66      0.63      0.61       494
  weighted avg       0.66      0.63      0.60       494

